**STYLE-MIXING NOTEBOOK**

--------------------------------------------------------------------------------

**KURULUM-AYAR VS.**

In [0]:
!pip install tensorflow-gpu==1.15
%rm -rf /content/sample_data
!git clone https://github.com/NVlabs/stylegan2.git
%cd stylegan2
%mkdir resim
%mkdir aligned_resim
%rm -rf /content/stylegan2/docs
%rm /content/stylegan2/Dockerfile
%rm /content/stylegan2/LICENSE.txt
%rm /content/stylegan2/README.md

In [0]:
%mkdir networks
%cd networks
!wget http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl
%cd ..

--------------------------------------------------------------------------------

**Resim indir-düzenle-göster-align**

In [0]:
!wget -O /content/stylegan2/resim/1.jpg 'https://external-preview.redd.it/lVUjjLxfqF7fIrqMslHnM8EVD6LQiO1yVnixSlco2W4.jpg?auto=webp&s=5d6c51f1be617680ebb5b64e4765c5d96269cec3'

In [0]:
from PIL import Image
im1 = Image.open(r'/content/stylegan2/resim/1.jpg')
im1.save(r'/content/stylegan2/resim/1.png')
%rm /content/stylegan2/resim/1.jpg

In [0]:
import PIL.Image
img1 = PIL.Image.open('/content/stylegan2/resim/1.png')
wpercent = (256/float(img1.size[0]))
hsize = int((float(img1.size[1])*float(wpercent)))
img1 = img1.resize((256,hsize), PIL.Image.LANCZOS)
display(img1)

In [0]:
#alıgnment kodu

import os, sys, bz2,  dlib, scipy.ndimage, PIL.Image
from keras.utils import get_file
import numpy as np


RAW_IMAGES_DIR = "/content/stylegan2/resim/"
ALIGNED_IMAGES_DIR = "/content/stylegan2/aligned_resim/"

LANDMARKS_MODEL_URL = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'


class LandmarksDetector:
    def __init__(self, predictor_model_path):
        """
        :param predictor_model_path: path to shape_predictor_68_face_landmarks.dat file
        """
        self.detector = dlib.get_frontal_face_detector() # cnn_face_detection_model_v1 also can be used
        self.shape_predictor = dlib.shape_predictor(predictor_model_path)

    def get_landmarks(self, image):
        img = dlib.load_rgb_image(image)
        dets = self.detector(img, 1)

        for detection in dets:
            face_landmarks = [(item.x, item.y) for item in self.shape_predictor(img, detection).parts()]
            yield face_landmarks


def image_align(src_file, dst_file, face_landmarks, output_size=1024, transform_size=4096, enable_padding=True):
        # Align function from FFHQ dataset pre-processing step
        # https://github.com/NVlabs/ffhq-dataset/blob/master/download_ffhq.py

        lm = np.array(face_landmarks)
        lm_chin          = lm[0  : 17]  # left-right
        lm_eyebrow_left  = lm[17 : 22]  # left-right
        lm_eyebrow_right = lm[22 : 27]  # left-right
        lm_nose          = lm[27 : 31]  # top-down
        lm_nostrils      = lm[31 : 36]  # top-down
        lm_eye_left      = lm[36 : 42]  # left-clockwise
        lm_eye_right     = lm[42 : 48]  # left-clockwise
        lm_mouth_outer   = lm[48 : 60]  # left-clockwise
        lm_mouth_inner   = lm[60 : 68]  # left-clockwise

        # Calculate auxiliary vectors.
        eye_left     = np.mean(lm_eye_left, axis=0)
        eye_right    = np.mean(lm_eye_right, axis=0)
        eye_avg      = (eye_left + eye_right) * 0.5
        eye_to_eye   = eye_right - eye_left
        mouth_left   = lm_mouth_outer[0]
        mouth_right  = lm_mouth_outer[6]
        mouth_avg    = (mouth_left + mouth_right) * 0.5
        eye_to_mouth = mouth_avg - eye_avg

        # Choose oriented crop rectangle.
        x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
        x /= np.hypot(*x)
        x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
        y = np.flipud(x) * [-1, 1]
        c = eye_avg + eye_to_mouth * 0.1
        quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
        qsize = np.hypot(*x) * 2

        # Load in-the-wild image.
        if not os.path.isfile(src_file):
            print('\nCannot find source image. Please run "--wilds" before "--align".')
            return
        img = PIL.Image.open(src_file)

        # Shrink.
        shrink = int(np.floor(qsize / output_size * 0.5))
        if shrink > 1:
            rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
            img = img.resize(rsize, PIL.Image.ANTIALIAS)
            quad /= shrink
            qsize /= shrink

        # Crop.
        border = max(int(np.rint(qsize * 0.1)), 3)
        crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
        if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
            img = img.crop(crop)
            quad -= crop[0:2]

        # Pad.
        pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
        if enable_padding and max(pad) > border - 4:
            pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
            img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
            h, w, _ = img.shape
            y, x, _ = np.ogrid[:h, :w, :1]
            mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
            blur = qsize * 0.02
            img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
            img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
            img = PIL.Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
            quad += pad[:2]

        # Transform.
        img = img.transform((transform_size, transform_size), PIL.Image.QUAD, (quad + 0.5).flatten(), PIL.Image.BILINEAR)
        if output_size < transform_size:
            img = img.resize((output_size, output_size), PIL.Image.ANTIALIAS)

        # Save aligned image.
        img.save(dst_file, 'PNG')



def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path


if __name__ == "__main__":
    landmarks_model_path = unpack_bz2(get_file('shape_predictor_68_face_landmarks.dat.bz2', LANDMARKS_MODEL_URL, cache_subdir='temp'))
    landmarks_detector = LandmarksDetector(landmarks_model_path)
    for img_name in os.listdir(RAW_IMAGES_DIR):
        raw_img_path = os.path.join(RAW_IMAGES_DIR, img_name)
        for i, face_landmarks in enumerate(landmarks_detector.get_landmarks(raw_img_path), start=1):
            face_img_name = '%s_%02d.png' % (os.path.splitext(img_name)[0], i)
            aligned_face_path = os.path.join(ALIGNED_IMAGES_DIR, face_img_name)
            image_align(raw_img_path, aligned_face_path, face_landmarks)


--------------------------------------------------------------------------------

**DATASET OLUŞTUR**

In [0]:
!python dataset_tool.py create_from_images Datasetler aligned_resim

--------------------------------------------------------------------------------

**PROJECT IMAGE-DLATENT OLUŞTUR**

```
# run_projector.py de project_image() fonksiyonuna
```

```
np.save(png_prefix + '-dlatent.npy', proj.get_dlatents())
np.save(png_prefix + '-noise.npy', proj.get_noises())
```

In [0]:
!python run_projector.py project-real-images --network=networks/stylegan2-ffhq-config-f.pkl --dataset=Datasetler --data-dir=. --num-images=1

In [0]:
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys

import pretrained_networks

def style_mixing(dlatent, col_seeds, truncation_psi, col_styles, minibatch_size=4):
    print('Loading networks from ...')
    _G, _D, Gs = pretrained_networks.load_networks('networks/stylegan2-ffhq-config-f.pkl')
    w_avg = Gs.get_var('dlatent_avg') # [component]
    row_seeds = [80]
    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = False
    Gs_syn_kwargs.minibatch_size = 1 #minibatch_size

    print('Generating W vectors...')
    all_seeds = list(set(col_seeds))
    all_z = np.stack([np.random.RandomState(seed).randn(*Gs.input_shape[1:]) for seed in all_seeds]) # [minibatch, component]

    all_w = Gs.components.mapping.run(all_z, None) # [minibatch, layer, component]
    all_w = np.concatenate((all_w, dlatent), 0)
    all_w = w_avg + (all_w - w_avg) * truncation_psi # [minibatch, layer, component]
    all_seeds.extend(row_seeds) #= list(set(col_seeds + row_seeds))
    w_dict = {seed: w for seed, w in zip(all_seeds, list(all_w))} # [layer, component]
    
    print('Generating images...')
    all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs) # [minibatch, height, width, channel]
    image_dict = {(seed, seed): image for seed, image in zip(all_seeds, list(all_images))}

    print('Generating style-mixed images...')
    for row_seed in row_seeds:
        for col_seed in col_seeds:
            w = w_dict[row_seed].copy()
            # We set col_styles in this part
            w[3:8] = w_dict[col_seed][3:8]
            image = Gs.components.synthesis.run(w[np.newaxis], **Gs_syn_kwargs)[0]
            image_dict[(row_seed, col_seed)] = image

    print('Saving images...')
    for (row_seed, col_seed), image in image_dict.items():
        PIL.Image.fromarray(image, 'RGB').save(dnnlib.make_run_dir_path('mix/'+'%d-%d.png' % (row_seed, col_seed)))

    print('Saving image grid...')
    _N, _C, H, W = Gs.output_shape
    canvas = PIL.Image.new('RGB', (W * (len(col_seeds) + 1), H * (len(row_seeds) + 1)), 'black')
    for row_idx, row_seed in enumerate([None] + row_seeds):
        for col_idx, col_seed in enumerate([None] + col_seeds):
            if row_seed is None and col_seed is None:
                continue
            key = (row_seed, col_seed)
            if row_seed is None:
                key = (col_seed, col_seed)
            if col_seed is None:
                key = (row_seed, row_seed)
            canvas.paste(PIL.Image.fromarray(image_dict[key], 'RGB'), (W * col_idx, H * row_idx))
    canvas.save(dnnlib.make_run_dir_path('mix/grid.png'))

--------------------------------------------------------------------------------

**STYLE-MIX**

In [0]:
%mkdir mix
dlat = np.load('/content/stylegan2/results/00000-project-real-images/image0000--dlatent.npy')
style_mixing(dlat, [1], 0.5, 4)

--------------------------------------------------------------------------------

**TÜM  RESİMLERİ VE GRIDLERİ GÖSTER**

In [0]:
display(PIL.Image.open('/content/stylegan2/mix/1-1.png').resize((256,256)))
display(PIL.Image.open('/content/stylegan2/mix/80-80.png').resize((256,256)))
display(PIL.Image.open('/content/stylegan2/mix/80-1.png').resize((256,256)))
print("------------------------------------------------------------------")
print("------------------------------------------------------------------")
print("------------------------------------------------------------------")

print("-------------------GRID----------------------")
display(PIL.Image.open('/content/stylegan2/mix/grid.png').resize((512,512)))

--------------------------------------------------------------------------------

**TEMİZLE**

In [0]:
%rm -rf /content/stylegan2/Datasetler
%rm -rf /content/stylegan2/aligned_resim
%rm -rf /content/stylegan2/networks
%rm -rf /content/stylegan2/resim
%rm -rf /content/stylegan2/results